In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from fastai import *        # Quick accesss to most common functionality
from fastai.text import *   # Quick accesss to NLP functionality
import html

In [6]:
from fastai import *

In [7]:
import torch
from pytorch_pretrained_bert import OpenAIGPTTokenizer#, OpenAIGPTModel, OpenAIGPTLMHeadModel
from gpt import OpenAIGPTModel, OpenAIGPTLMHeadModel

In [8]:
def process_all(self, texts):
    return [self.tokenize(x) for x in texts]

# def numericalize(self, tokenized_text):
#     return self.convert_tokens_to_ids(tokenized_text)

# def textify(self, ids):
#     return tokenizer.convert_ids_to_tokens

OpenAIGPTTokenizer.process_all = process_all
OpenAIGPTTokenizer.numericalize = OpenAIGPTTokenizer.convert_tokens_to_ids
OpenAIGPTTokenizer.textify = OpenAIGPTTokenizer.convert_ids_to_tokens

# Load pre-trained model tokenizer (vocabulary)
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')

# Tokenized input

text = "Who was Jim Henson ? Jim Henson was a puppeteer"
tokenized_text = tokenizer.tokenize(text)

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens]).cuda()

In [9]:
PATH=Path('data/wikitext-2-raw')

In [10]:
def read_file(fn:PathOrStr, enc='utf-8'):
    "Read the text in `fn`."
#     with open(fn,'r', encoding = enc) as f: return ''.join(f.read().splitlines())
    tokens = []
    with open(fn,'r', encoding = enc) as f: 
        for line in f.read().splitlines():
            l = line.strip()
            if len(l) == 0: continue
            tokens.append(l)#.split())
    return np.array(tokens)[:1000]

In [11]:
train_tok = read_file(PATH/'wiki.train.raw')
valid_tok = read_file(PATH/'wiki.valid.raw')
test_tok = read_file(PATH/'wiki.test.raw')

In [12]:
class GPTTokenizeProcessor(PreProcessor):
    "`PreProcessor` that tokenizes the texts in `ds`."
    def __init__(self, ds:ItemList=None, tokenizer:Tokenizer=None, chunksize:int=10000):
        self.tokenizer,self.chunksize = ifnone(tokenizer, Tokenizer()),chunksize

    def process_one(self, item):  return self.tokenizer._process_all_1([item])[0]
    def process(self, ds):
#         ds.items = _join_texts(ds.items, self.mark_fields)
        tokens = []
        for i in progress_bar(range(0,len(ds),self.chunksize), leave=False):
            tokens += self.tokenizer.process_all(ds.items[i:i+self.chunksize])
        ds.items = tokens

In [13]:
p = [GPTTokenizeProcessor(chunksize=1000, tokenizer=tokenizer), 
     NumericalizeProcessor(vocab=tokenizer)]

In [14]:
bptt = 128
bs=8

In [15]:
path = PATH
# processor = data._get_processor(tokenizer=tok, vocab=None, max_vocab=60000)
# processor = data._get_processor(vocab=None, chunksize=500)
src = ItemLists(path, TextList(train_tok, path=path, processor=p),
                TextList(valid_tok, path=path, processor=p))
src = src.label_for_lm()
text_data = src.databunch(bs=bs,bptt=bptt)

In [16]:
text_data.train_ds[1]

(Text ['sen', 'j', 'ō</w>', 'no</w>', 'val', 'ky', 'ria</w>', '3</w>', ':</w>', 'un', 'recorded</w>', 'chronicles</w>', '(</w>', 'japanese</w>', ':</w>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '3</w>', ',</w>', 'lit</w>', '.</w>', 'val', 'ky', 'ria</w>', 'of</w>', 'the</w>', 'battlefield</w>', '3</w>', ')</w>', ',</w>', 'commonly</w>', 'referred</w>', 'to</w>', 'as</w>', 'val', 'ky', 'ria</w>', 'chronicles</w>', 'iii</w>', 'outside</w>', 'japan</w>', ',</w>', 'is</w>', 'a</w>', 'tactical</w>', 'role</w>', '@</w>', '-</w>', '@</w>', 'playing</w>', 'video</w>', 'game</w>', 'developed</w>', 'by</w>', 'se', 'ga</w>', 'and</w>', 'media</w>', '.</w>', 'vision</w>', 'for</w>', 'the</w>', 'play', 'station</w>', 'portable</w>', '.</w>', 'released</w>', 'in</w>', 'january</w>', '2011</w>', 'in</w>', 'japan</w>', ',</w>', 'it</w>', 'is</w>', 'the</w>', 'third</w>', 'game</w>', 'in</w>', 'the</w>', 'val', 'ky', 'ria</w>', 'series</w>', '.</w>', 'e

In [29]:
class GPTLoss():
    def __init__(self, model):
        self.model = model
        self.loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-1)
    def forward(self, lm_logits, lm_labels):
        loss = self.loss_fct(lm_logits.view(-1, lm_logits.size(-1)), lm_labels.view(-1))
        return loss
    def __call__(self, lm_logits, lm_labels):
        loss = self.loss_fct(lm_logits.view(-1, lm_logits.size(-1)), lm_labels.view(-1))
        return loss

In [18]:
# Load pre-trained model (weights)
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt').cuda()
# model.eval()

# Predict all tokens
# predictions = model(tokens_tensor)

In [19]:
learn = LanguageLearner(text_data, model, bptt)

In [30]:
learn.loss_func = GPTLoss(model)

In [21]:
learn.callbacks = []

In [31]:
learn.fit_one_cycle(2, 1e-4, moms=(0.7,0.5))

epoch,train_loss,valid_loss,accuracy
1,1.837704,5.458471,0.278564
2,1.450587,5.306539,0.281860


In [32]:
learn.fit_one_cycle(2, 1e-4, moms=(0.7,0.5))

epoch,train_loss,valid_loss,accuracy
1,1.344717,5.754536,0.283325
2,1.033213,5.553970,0.275513


In [33]:
learn.fit_one_cycle(2, 1e-5, moms=(0.7,0.5))

epoch,train_loss,valid_loss,accuracy
1,3.137744,4.836852,0.290771
2,3.285836,4.576702,0.297974


In [24]:
%debug

> /home/ubuntu/fastai/fastai/basic_train.py(23)loss_batch()
     21 
     22     if not loss_func: return to_detach(out), yb[0].detach()
---> 23     loss = loss_func(out, *yb)
     24 
     25     if opt is not None:

ipdb> loss_func
functools.partial(<class '__main__.GPTLoss'>, OpenAIGPTLMHeadModel(
  (transformer): OpenAIGPTModel(
    (embed): Embedding(40990, 768)
    (drop): Dropout(p=0.1)
    (h): ModuleList(
      (0): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1)
          (resid_dropout): Dropout(p=0.1)
        )
        (ln_1): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1)
        )
        (ln_2): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
      )
      (1): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          